# We create the tables
Execution of the first step: Drop the table if exists and create the tables.

In [1]:
!python 1_create_tables.py

DROP TABLE IF EXISTS staging_events 

DROP TABLE IF EXISTS staging_songs 

DROP TABLE IF EXISTS fact_songplay 

DROP TABLE IF EXISTS dim_users 

DROP TABLE IF EXISTS dim_songs 

DROP TABLE IF EXISTS dim_artists 

DROP TABLE IF EXISTS dim_time 

CREATE TABLE staging_events (
                                  artist text,
                                  auth text,
                                  first_name text,
                                  gender text,
                                  itemInSession int,
                                  lastName text,
                                  lenght DOUBLE PRECISION,
                                  level text,
                                  location text,
                                  method text,
                                  page text,
                                  registration bigint,
                                  sessionId int,
                                  song text,
                                  stat

# Run the ETL
The second step is take the data into the staging tables and later insert it into the star-schema tables.

In [2]:
!python 2_etl.py

Loading data into staging tables...
copy staging_events 
                           from 's3://udacity-dend/log_data'
                           credentials 'aws_iam_role=arn:aws:iam::276425435005:role/dwhRole'
                           json 's3://udacity-dend/log_json_path.json' 
                           region 'us-west-2';
                        
copy staging_songs from 's3://udacity-dend/song_data'
                           credentials 'aws_iam_role=arn:aws:iam::276425435005:role/dwhRole'
                           json 'auto ignorecase'
                           region 'us-west-2';
                        
Inserting data into definitive tables...
INSERT INTO dim_users (user_id, level, first_name, last_name, gender)
                        Select distinct
                            userid,
                            level,
                            first_name,
                            lastname,
                            gender
                        FROM staging_even

# Analysis
Finally, we execute some additional queries to check the data is correct.

In [7]:
!python 3_analysis.py

Checking the number of rows of each table...
SELECT COUNT(*) from staging_events
[(8056,)]

SELECT COUNT(*) from staging_songs
[(14896,)]

SELECT COUNT(*) from dim_time
[(6813,)]

SELECT COUNT(*) from dim_users
[(105,)]

SELECT COUNT(*) from dim_artists
[(10025,)]

SELECT COUNT(*) from dim_songs
[(14896,)]

SELECT COUNT(*) from fact_songplay
[(6820,)]

Solving some example questions...
1. How many active users are there in the app?
Select count(DISTINCT USER_ID) FROM DIM_USERS
(97,)

2. What is the location with more reproductions?

            Select location, count(*) 
            from fact_songplay
            group by location
            order by 2 desc
            limit 1
('San Francisco-Oakland-Hayward, CA', 691)

3. Build a ranking with the users with more reproductions in the app

            SELECT
                f.user_id, u.first_name, u.last_name, count(*)
            FROM
                fact_songplay f
                inner join dim_users u on u.user_id = f.user_id
    